# NerF + QG Loss

The full QG equation is given by:

$$
\begin{aligned}
\partial_t q + \det \boldsymbol{J}(q, \psi) &= 0
\end{aligned}
$$

where:

* $q=\nabla^2 \psi$
* $\det \boldsymbol{J}(q, \psi)=\partial_x q\partial_y\psi - \partial_y q\partial_x\psi$.

We are interested in finding some NerF method that can take in the spatial-temporal coordinates, $\mathbf{x}_\phi$, and output a vector corresponding to the PV and stream function, $\psi$, i.e. $\mathbf{y}_\text{obs}$.

$$
\mathbf{y}_\text{obs} = \boldsymbol{f_\theta}(\mathbf{x}_\phi) + \epsilon, \hspace{5mm}\epsilon \sim \mathcal{N}(0, \sigma^2)
$$

We use a SIREN network which is a fully connected neural network with the $sin$ activation function.

* **Data Inputs**: `256x256x11`
* **Data Ouputs**: `2`


In [ ]:
import sys, os
from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".root"])

# append to path
sys.path.append(str(root))

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from ml_collections import config_dict
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger

pl.seed_everything(123)

import matplotlib.pyplot as plt
import seaborn as sns
import wandb
from config_1 import get_config


sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# from ml_collections import config_dict

# cfg = config_dict.ConfigDict()

# # logging args
# cfg.log = config_dict.ConfigDict()
# cfg.log.mode = "online" #"disabled"
# cfg.log.project ="inr4ssh"
# cfg.log.entity = "ige"
# cfg.log.log_dir = "/Users/eman/code_projects/logs/"
# cfg.log.resume = False

# # data args
# cfg.data = config_dict.ConfigDict()
# cfg.data.data_dir =  f"/Users/eman/code_projects/torchqg/data/qgsim_simple_128x128.nc"

# # preprocessing args
# cfg.pre = config_dict.ConfigDict()
# cfg.pre.noise = 0.01
# cfg.pre.dt = 1.0
# cfg.pre.time_min = 500
# cfg.pre.time_max = 511
# cfg.pre.seed = 123

# # train/test args
# cfg.split = config_dict.ConfigDict()
# cfg.split.train_prct = 0.9

# # dataloader args
# cfg.dl = config_dict.ConfigDict()
# cfg.dl.batchsize_train = 2048
# cfg.dl.batchsize_val = 1_000
# cfg.dl.batchsize_test = 5_000
# cfg.dl.batchsize_predict = 10_000
# cfg.dl.num_workers = 0
# cfg.dl.pin_memory = False

# # loss arguments
# cfg.loss = config_dict.ConfigDict()
# cfg.loss.qg = True
# cfg.loss.alpha = 1e-4

# # optimizer args
# cfg.optim = config_dict.ConfigDict()
# cfg.optim.warmup = 10
# cfg.optim.num_epochs = 100
# cfg.optim.learning_rate = 1e-4

# # trainer args
# cfg.trainer = config_dict.ConfigDict()
# cfg.trainer.accelerator = None
# cfg.trainer.devices = 1
# cfg.trainer.grad_batches = 1

In [ ]:
from inr4ssh._src.io import transform_dict

cfg = get_config()

cfg.to_dict()

In [ ]:
wandb_logger = WandbLogger(
    config=cfg.to_dict(),
    mode="offline",  # cfg.log.mode,
    project=cfg.log.project,
    entity=cfg.log.entity,
    dir=cfg.log.log_dir,
    resume=False,
)

In [ ]:
!ls /Users/eman/code_projects/torchqg/data/

## Data Module

Now we will put all of the preprocessing routines together. This is **very important** for a few reasons:

1. It collapses all of the operations in a modular way
2. It makes it reproducible for the next people
3. It makes it very easy for the PyTorch-Lightning framework down the line.

In [ ]:
from experiments.qg.qg_sim.model import INRModel
from experiments.qg.qg_sim.data import QGSimulation

In [ ]:
dm = QGSimulation(cfg)
dm.setup()

In [ ]:
x_init, y_init = dm.ds_train[:10]

In [ ]:
x_init.shape, y_init.shape

## NerF

This standard Neural Fields.

In [ ]:
from inr4ssh._src.models.siren import Siren, SirenNet

In [ ]:
dim_in = x_init.shape[1]
dim_hidden = 256
dim_out = y_init.shape[1]
num_layers = 4
w0 = 1.0
w0_initial = 30.0
c = 6.0
final_activation = None  # nn.Sigmoid()

net = SirenNet(
    dim_in=dim_in,
    dim_hidden=dim_hidden,
    dim_out=dim_out,
    num_layers=num_layers,
    w0=w0,
    w0_initial=w0_initial,
    c=c,
    final_activation=final_activation,
)

In [ ]:
out = net(x_init)

## PINNS Loss

$$
\partial_t \nabla^2 \psi + \det J(\psi, \nabla^2\psi) = 0
$$

In [ ]:
from experiments.qg.qg_sim.losses import RegQG, initialize_data_loss

In [ ]:
reg_loss = RegQG(1e-4)

In [ ]:
reg_loss(x_init, net)

## Experiment

In [ ]:
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from functools import partial
from typing import Dict, Any, cast
from torch.optim import Adam

In [ ]:
run_path = "ige/inr4ssh/1st3rtl0"
model_path = "checkpoints/epoch=990-step=39640.ckpt"

In [ ]:
old_config = get_wandb_config(run_path)
from ml_collections import config_dict

In [ ]:
learn.cpu()

In [ ]:
from inr4ssh._src.io import get_wandb_config, get_wandb_model

In [ ]:
best_model = get_wandb_model(run_path, model_path)
best_model.download(replace=True)

### Callbacks

In [ ]:
from pytorch_lightning.callbacks import TQDMProgressBar, ModelCheckpoint

In [ ]:
old_config = get_wandb_config(config.pretrained.run_path)

In [ ]:
from pathlib import Path

In [ ]:
model_cb = ModelCheckpoint(
    dirpath=str(Path(wandb_logger.experiment.dir).joinpath("checkpoints")),
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

In [ ]:
callbacks = [model_cb, TQDMProgressBar(refresh_rate=1)]

### Learner

In [ ]:
state = torch.load(best_model.name, map_location=torch.device("cpu"))

In [ ]:
# state

In [ ]:
wandb_logger.experiment.

In [ ]:
# state["state_dict"]

In [ ]:
kwargs,
net = SirenNet(**kwargs)
net.load_state_dict(state_dict)

In [ ]:
from experiments.qg.qg_sim.model import INRModel

In [ ]:
learn = INRModel.load_from_checkpoint(
    best_model.name,
    model=net,
    loss_data=nn.MSELoss("mean"),
    reg_pde=reg_loss,
    learning_rate=cfg.optim.learning_rate,
    warmup=cfg.optim.warmup,
    num_epochs=cfg.optim.num_epochs,
    alpha=cfg.loss.alpha,
    qg=cfg.loss.qg,
)

### Trainer

In [ ]:
trainer = Trainer(
    min_epochs=1,
    max_epochs=cfg.optim.num_epochs,
    accelerator=cfg.trainer.accelerator,
    devices=cfg.trainer.devices,
    enable_progress_bar=True,
    logger=wandb_logger,
    callbacks=callbacks,
    accumulate_grad_batches=cfg.trainer.grad_batches,
)

### Train

In [ ]:
trainer.fit(
    learn,
    datamodule=dm,
)

## Results

### Testing

In [ ]:
# res = trainer.test(learn, dataloaders=dm.test_dataloader())

# results["data"] = res

In [ ]:
import wandb

wandb.finish()

### Predictions

In [ ]:
# t0 = time.time()
predictions = trainer.predict(learn, datamodule=dm, return_predictions=True)
predictions = torch.cat(predictions)
# t1 = time.time() - t0

In [ ]:
ds_pred = dm.create_predictions_ds(predictions)

In [ ]:
from inr4ssh._src.operators import differential_simp as diffops_simp
from inr4ssh._src.operators import differential as diffops

In [ ]:
# ds_pred = dm.create_predictions_ds(predictions)
# ds_pred

In [ ]:
import tqdm

In [ ]:
learn.model.eval()
coords, truths, preds, grads, qs = [], [], [], [], []
for ix, iy in tqdm.tqdm(dm.predict_dataloader()):
    with torch.set_grad_enabled(True):
        # prediction
        ix = torch.autograd.Variable(ix.clone(), requires_grad=True)
        p_pred = learn.model(ix)

        # p_pred = p_pred.clone()
        # p_pred.require_grad_ = True

        # gradient
        p_grad = diffops_simp.gradient(p_pred, ix)
        # p_grad = diffops.grad(p_pred, ix)
        # q
        q = diffops_simp.divergence(p_grad, ix)
        # q = diffops.div(p_grad, ix)

    # collect
    truths.append(iy)
    coords.append(ix)
    preds.append(p_pred)
    grads.append(p_grad)
    qs.append(q)

coords = torch.cat(coords).detach().numpy()
preds = torch.cat(preds).detach().numpy()
truths = torch.cat(truths).detach().numpy()
grads = torch.cat(grads).detach().numpy()
qs = torch.cat(qs).detach().numpy()

In [ ]:
df_data = dm.create_predictions_df()

np.testing.assert_array_almost_equal(coords, df_data[["Nx", "Ny", "steps"]])
np.testing.assert_array_almost_equal(truths, df_data[["p"]])

df_data["p_pred"] = preds
df_data["u_pred"] = grads[:, 0]
df_data["v_pred"] = grads[:, 1]
df_data["q_pred"] = qs

xr_data = df_data.set_index(["Nx", "Ny", "steps"]).to_xarray()

### Figure I: Stream Function

In [ ]:
xr_data.p_pred.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

xr_data.p.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)
np.abs(xr_data.p_pred - xr_data.p).thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

In [ ]:
# ds_pred.pred.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### Figure II: PV

In [ ]:
xr_data.q_pred.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

xr_data.q.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)
np.abs(xr_data.q_pred - xr_data.q).thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

In [ ]:
ds_pred.true.hvplot.image(x="Nx", y="Ny", width=500, height=400, cmap="viridis")

### Figure III: U Velocity

In [ ]:
xr_data.u_pred.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

xr_data.u.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)
np.abs(xr_data.u_pred - xr_data.u).thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

In [ ]:
# (ds_pred.true - ds_pred.pred).hvplot.image(
#     x="Nx", y="Ny", width=500, height=400, cmap="viridis"
# )

### Figure IV: V Velocity

In [ ]:
xr_data.v_pred.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)

xr_data.v.thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)
np.abs(xr_data.v_pred - xr_data.v).thin(steps=1).plot.imshow(
    col="steps",
    robust=True,
    col_wrap=3,
    cmap="viridis",
)